In [1]:
from app.tools.s3 import validate_path, client, bucket
from app.tools.db import  DatabaseConnection
from app.tools.constans import paramsl, paramsp
from app.match import match_img
from pprint import pprint as pp
import Levenshtein as lev
import ijson, json, os
import pandas as pd
import glob
import uuid
import re

path_input = "trash/dicts/old_matches.json"
conn = DatabaseConnection(connect= True, **paramsl)
conn1 = DatabaseConnection(connect= True, **paramsl)
conn2 = DatabaseConnection(connect= True, **paramsp)

def get_path(files: str, paths: str) -> str:
    for file in files:
        if file != None:
            path_files = [validate_path(os.path.join(path,file)) for path in paths]
            if any(path_files):
                return paths[path_files.index(True)]
    else:
        return None

c:\Users\Administrator\Desktop\match2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
request = {
    "request_id": "e07aaf8f-a587-42af-a0b3-0abd42a7ffc5",
    "input":{
                "origin" : {"products":{"brand": "ARRABI"}},
                "alternative" : {"products":{"brand": "Arrabi"}}
            },
    "s3_path_img_origin": "ajio-myntra/origin/20240131/",
    "s3_path_img_alternative": "ajio-myntra/alternative/20240202/dev/"
}

match_img(**request)

SELECT id, products FROM request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5.origin WHERE (products->> 'brand' = 'ARRABI')
La conexión fue cerrada. No se puede hacer commit
SELECT id, products FROM request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5.alternative WHERE (products->> 'brand' = 'Arrabi')
La conexión fue cerrada. No se puede hacer commit
False
False
